# setup

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Sep 14 14:33:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
cd /content/drive/MyDrive/muss

/content/drive/MyDrive/muss


In [ ]:
pip install -e . &> /dev/null

In [ ]:
!python -m spacy download en_core_web_md &> /dev/null

In [ ]:
!pip install sacrebleu==1.5.1

     |████████████████████████████████| 54 kB 2.5 MB/s 
  Attempting uninstall: portalocker
    Found existing installation: portalocker 2.3.2
    Uninstalling portalocker-2.3.2:
      Successfully uninstalled portalocker-2.3.2
  Attempting uninstall: sacrebleu
    Found existing installation: sacrebleu 2.0.0
    Uninstalling sacrebleu-2.0.0:
      Successfully uninstalled sacrebleu-2.0.0


In [ ]:
!pip install sacrebleu==1.5.1

# train

In [ ]:
!pip install fairseq==0.10.2

     |████████████████████████████████| 1.7 MB 9.5 MB/s 
     |████████████████████████████████| 145 kB 82.4 MB/s 
     |████████████████████████████████| 74 kB 4.1 MB/s 
     |████████████████████████████████| 112 kB 95.3 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=d00d66f85c102eba81e57fc5c98779ef994097409ece14a251f617fb27bca07b
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime


In [ ]:
# muss/scripts/train_model.py

from muss.fairseq.main import fairseq_train_and_evaluate_with_parametrization
from muss.mining.training import get_score_rows #get_bart_kwargs

#########################
import re
import shutil

from cachier import cachier
from easse.cli import evaluate_system_output
from easse.utils.constants import TEST_SETS_PATHS
import torch
from tqdm import tqdm

from muss.resources.paths import get_data_filepath, MODELS_DIR, get_dataset_dir
from muss.utils.helpers import add_dicts, args_str_to_dict
from muss.utils.resources import download_and_extract
from muss.preprocessors import GPT2BPEPreprocessor
from muss.preprocessing import apply_line_function_to_file
from muss.fairseq.main import get_language_from_dataset
from muss.text import truncate


INFO:root:Generating grammar tables from /usr/lib/python3.7/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.7/lib2to3/PatternGrammar.txt


In [ ]:
gpt2_prepro = GPT2BPEPreprocessor()

In [ ]:
encode = gpt2_prepro.encode_sentence('<NEXT_DIFFICULT_WORD> <NEXT_NE> comprehend <SENT_START> This is extremely hard to comprehend.')
encode

encoder_json_path /content/drive/MyDrive/muss/resources/bart_bpe/encoder.json
vocab_bpe_path /content/drive/MyDrive/muss/resources/bart_bpe/vocab.bpe


'173 220 124 24772 220 125 770 318 4457 1327 284 24772 13'

In [ ]:
gpt2_prepro.decode_sentence(encode)

'<NEXT_DIFFICULT_WORD> <NEXT_NE> comprehend <SENT_START> This is extremely hard to comprehend.'

In [ ]:
def get_evaluate_kwargs(language, phase='valid'):
    return {
        ('en', 'valid'): {'test_set': 'asset_valid'},
        ('en', 'test'): {'test_set': 'asset_test'},
        ('fr', 'valid'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('alector', 'valid', 'complex'),
            'refs_sents_paths': [get_data_filepath('alector', 'valid', 'simple')],
        },
        ('fr', 'test'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('alector', 'test', 'complex'),
            'refs_sents_paths': [get_data_filepath('alector', 'test', 'simple')],
        },
        ('es', 'valid'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('simplext_corpus', 'valid', 'complex'),
            'refs_sents_paths': [get_data_filepath('simplext_corpus', 'valid', 'simple')],
        },
        ('es', 'test'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('simplext_corpus', 'test', 'complex'),
            'refs_sents_paths': [get_data_filepath('simplext_corpus', 'test', 'simple')],
        },
    }[(language, phase)]

evaluate_kwargs=get_evaluate_kwargs('en')
evaluate_kwargs

{'test_set': 'asset_valid'}

In [ ]:
def get_predict_files(language):
    return {
        'en': [get_data_filepath('asset', 'valid', 'complex'), get_data_filepath('asset', 'test', 'complex')],
        'fr': [get_data_filepath('alector', 'valid', 'complex'), get_data_filepath('alector', 'test', 'complex')],
        'es': [
            get_data_filepath('simplext_corpus', 'valid', 'complex'),
            get_data_filepath('simplext_corpus', 'test', 'complex'),
        ],
    }[language]


In [ ]:

def get_access_preprocessors_kwargs(language, use_short_name=False):
    return {
        'LengthRatioPreprocessor': {'target_ratio': 0.8, 'use_short_name': use_short_name},
        'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.8, 'use_short_name': use_short_name},
        'WordRankRatioPreprocessor': {'target_ratio': 0.8, 'language': language, 'use_short_name': use_short_name},
        'DependencyTreeDepthRatioPreprocessor': {
            'target_ratio': 0.8,
            'language': language,
            'use_short_name': use_short_name,
        },
    }

In [ ]:

def get_bart_kwargs(dataset, language, use_access, use_short_name=False, bart_model='bart.large'):
    assert language == 'en'

    bart_path = '/content/drive/MyDrive/muss/resources/models/bart_mined/model.pt'
    arch = {
        'bart.base': 'bart_base',
        'bart.large': 'bart_large',
        'bart.large.cnn': 'bart_large',
    }[bart_model]
    kwargs = {
        'dataset': dataset,
        'metrics_coefs': [0, 1, 0],
        'parametrization_budget': 128,
        'predict_files': get_predict_files(language), # used in fairseq_evaluate_and_save
        'preprocessors_kwargs': {
            'GPT2BPEPreprocessor': {},
        },
        'preprocess_kwargs': {'dict_path': GPT2BPEPreprocessor().dict_path},
        'train_kwargs': {
            'ngpus': 8,
            'arch': arch,
            'restore_file': bart_path,
            'max_tokens': 4096,
            'lr': 3e-05,
            'warmup_updates': 500,
            'truncate_source': True,
            'layernorm_embedding': True,
            'share_all_embeddings': True,
            'share_decoder_input_output_embed': True,
            'reset_optimizer': True,
            'reset_dataloader': True,
            'reset_meters': True,
            'required_batch_size_multiple': 1,
            'criterion': 'label_smoothed_cross_entropy',
            'label_smoothing': 0.1,
            'dropout': 0.1,
            'attention_dropout': 0.1,
            'weight_decay': 0.01,
            'optimizer': 'adam',
            'adam_betas': '(0.9, 0.999)',
            'adam_eps': 1e-08,
            'clip_norm': 0.1,
            'lr_scheduler': 'polynomial_decay',
            'max_update': 20000,
            'skip_invalid_size_inputs_valid_test': True,
            'find_unused_parameters': True,
        },
        'evaluate_kwargs': {'test_set': 'asset_valid_NER_ABCD'},
    }
    if use_access:
        kwargs['preprocessors_kwargs'] = add_dicts(
            get_access_preprocessors_kwargs(language, use_short_name=use_short_name), kwargs['preprocessors_kwargs']
        )
    return kwargs

In [ ]:
# training dataset 
dataset = '0913_ABCD_NER_wikilarge'
kwargs = get_bart_kwargs(dataset=dataset, language='en', use_access=True)
kwargs['train_kwargs']['ngpus'] = 1  # Set this from 8 to 1 for local training
kwargs['train_kwargs']['max_tokens'] = 1024  # Lower this number to prevent OOM


In [ ]:
kwargs

{'dataset': '0913_ABCD_NER_wikilarge',
 'evaluate_kwargs': {'test_set': 'asset_valid_NER_ABCD'},
 'metrics_coefs': [0, 1, 0],
 'parametrization_budget': 128,
 'predict_files': [PosixPath('/content/drive/MyDrive/muss/resources/datasets/asset/valid.complex'),
  PosixPath('/content/drive/MyDrive/muss/resources/datasets/asset/test.complex')],
 'preprocess_kwargs': {'dict_path': PosixPath('/content/drive/MyDrive/muss/resources/bart_bpe/dict.txt')},
 'preprocessors_kwargs': {'DependencyTreeDepthRatioPreprocessor': {'language': 'en',
   'target_ratio': 0.8,
   'use_short_name': False},
  'GPT2BPEPreprocessor': {},
  'LengthRatioPreprocessor': {'target_ratio': 0.8, 'use_short_name': False},
  'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.8,
   'use_short_name': False},
  'WordRankRatioPreprocessor': {'language': 'en',
   'target_ratio': 0.8,
   'use_short_name': False}},
 'train_kwargs': {'adam_betas': '(0.9, 0.999)',
  'adam_eps': 1e-08,
  'arch': 'bart_large',
  'attention_dropout

In [ ]:
result = fairseq_train_and_evaluate_with_parametrization(**kwargs)

fairseq_prepare_and_train...



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/content/drive/MyDrive/muss/muss/utils/helpers.py", line 384, in __enter__
    sys.settrace(lambda *args, **keys: None)

INFO:fairseq_cli.train:Namespace(activation_fn='gelu', adam_betas='(0.9, 0.999)', adam_eps=1e-08, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='bart_large', attention_dropout=0.1, batch_size=64, batch_size_valid=64, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.1, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='/content/drive/MyDrive/m

exp_dir=/content/drive/MyDrive/muss/experiments/fairseq/local_1631567843618
train seed: 436
distributed_port 18969
fairseq-train /content/drive/MyDrive/muss/resources/datasets/_54645c8788bc40a6de008c8a68cd4a61/fairseq_preprocessed_complex-simple --task translation --source-lang complex --target-lang simple --save-dir /content/drive/MyDrive/muss/experiments/fairseq/local_1631567843618/checkpoints --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 --criterion label_smoothed_cross_entropy --label-smoothing 0.1 --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 500 --update-freq 128 --arch bart_large --dropout 0.1 --weight-decay 0.0 --clip-norm 0.1 --share-all-embeddings --save-interval 1 --validate-interval 1 --max-update 20000 --keep-interval-updates 1 --validate-interval-updates 100 --patience 5 --batch-size 64 --seed 436 --distributed-world-size 1 --distributed-port 18969 --tensorboard-logdir /content/drive/MyDrive/muss/experiments/fairseq/local_1631567843618/tensorbo

INFO:fairseq.tasks.translation:[complex] dictionary: 50264 types
INFO:fairseq.tasks.translation:[simple] dictionary: 50264 types
INFO:fairseq.data.data_utils:loaded 418 examples from: /content/drive/MyDrive/muss/resources/datasets/_54645c8788bc40a6de008c8a68cd4a61/fairseq_preprocessed_complex-simple/valid.complex-simple.complex
INFO:fairseq.data.data_utils:loaded 418 examples from: /content/drive/MyDrive/muss/resources/datasets/_54645c8788bc40a6de008c8a68cd4a61/fairseq_preprocessed_complex-simple/valid.complex-simple.simple
INFO:fairseq.tasks.translation:/content/drive/MyDrive/muss/resources/datasets/_54645c8788bc40a6de008c8a68cd4a61/fairseq_preprocessed_complex-simple valid complex-simple 418 examples
INFO:fairseq_cli.train:BARTModel(
  (encoder): TransformerEncoder(
    (dropout_module): FairseqDropout()
    (embed_tokens): Embedding(50264, 1024, padding_idx=1)
    (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
    (layernorm_embedding): LayerNorm((1024,), e

fairseq_prepare_and_train completed after 42126.91s.
find_best_parametrization...
| IN find_best_parametrization_nevergrad: kwargs.get('generate_kwargs', {}) {}
get ref from ( evaluate_simplifier) asset_valid_NER_ABCD
test_set is asset_valid_NER_ABCD
find_best_parametrization failed after 0.06s.


AssertionError: ignored

# find ACCESS parameter

In [ ]:
exp_dir ='/content/drive/MyDrive/muss/experiments/fairseq/local_1631567843618/'

In [ ]:
# dataset is changed here
kwargs['evaluate_kwargs'] = {'test_set': 'asset_valid_NER_ABCD'}

In [ ]:
from muss.fairseq.main import find_best_parametrization
recommended_preprocessors_kwargs = find_best_parametrization(exp_dir, **kwargs)
print(f'recommended_preprocessors_kwargs={recommended_preprocessors_kwargs}')
kwargs['preprocessor_kwargs'] = recommended_preprocessors_kwargs

| IN find_best_parametrization_nevergrad: kwargs.get('generate_kwargs', {}) {}
get ref from ( evaluate_simplifier) asset_valid_NER_ABCD
test_set in evaluate_simplifier when get ori asset_valid
encoder_json_path /content/drive/MyDrive/muss/resources/bart_bpe/encoder.json
vocab_bpe_path /content/drive/MyDrive/muss/resources/bart_bpe/vocab.bpe
| use checkpoint_paths [PosixPath('/content/drive/MyDrive/muss/experiments/fairseq/local_1631567843618/model.pt')]


INFO:fairseq_cli.generate:Namespace(all_gather_list_size=16384, batch_size=None, batch_size_valid=None, beam=5, bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', constraints=None, cpu=False, criterion='cross_entropy', curriculum=0, data='/tmp/tmp3sx_0tqe', data_buffer_size=10, dataset_impl='raw', ddp_backend='c10d', decoding_format=None, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_port=-1, distributed_rank=0, distributed_world_size=1, distributed_wrapper='DDP', diverse_beam_groups=-1, diverse_beam_strength=0.5, diversity_rate=-1.0, empty_cache_freq=0, eval_bleu=False, eval_bleu_args=None, eval_bleu_detok='space', eval_bleu_detok_args=None, eval_bleu_print_samples=False, eval_bleu_remove_bpe=None, eval_tokenized_bleu=False, fast_stat_sync=False, find_unused_parameters=False, fix_batches_to_gpus=False, fixed_validation_seed=None, forc

| IN find_best_parametrization_nevergrad: kwargs.get('generate_kwargs', {}) {}
get ref from ( evaluate_simplifier) asset_valid_NER_ABCD
test_set in evaluate_simplifier when get ori asset_valid
encoder_json_path /content/drive/MyDrive/muss/resources/bart_bpe/encoder.json
vocab_bpe_path /content/drive/MyDrive/muss/resources/bart_bpe/vocab.bpe


INFO:fairseq_cli.generate:Namespace(all_gather_list_size=16384, batch_size=None, batch_size_valid=None, beam=5, bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', constraints=None, cpu=False, criterion='cross_entropy', curriculum=0, data='/tmp/tmp645srnkb', data_buffer_size=10, dataset_impl='raw', ddp_backend='c10d', decoding_format=None, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_port=-1, distributed_rank=0, distributed_world_size=1, distributed_wrapper='DDP', diverse_beam_groups=-1, diverse_beam_strength=0.5, diversity_rate=-1.0, empty_cache_freq=0, eval_bleu=False, eval_bleu_args=None, eval_bleu_detok='space', eval_bleu_detok_args=None, eval_bleu_print_samples=False, eval_bleu_remove_bpe=None, eval_tokenized_bleu=False, fast_stat_sync=False, find_unused_parameters=False, fix_batches_to_gpus=False, fixed_validation_seed=None, forc

| use checkpoint_paths [PosixPath('/content/drive/MyDrive/muss/experiments/fairseq/local_1631567843618/model.pt')]


...

| IN find_best_parametrization_nevergrad: kwargs.get('generate_kwargs', {}) {}
get ref from ( evaluate_simplifier) asset_valid_NER_ABCD
test_set in evaluate_simplifier when get ori asset_valid
encoder_json_path /content/drive/MyDrive/muss/resources/bart_bpe/encoder.json
vocab_bpe_path /content/drive/MyDrive/muss/resources/bart_bpe/vocab.bpe


INFO:fairseq_cli.generate:Namespace(all_gather_list_size=16384, batch_size=None, batch_size_valid=None, beam=5, bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', constraints=None, cpu=False, criterion='cross_entropy', curriculum=0, data='/tmp/tmpwnzo6byo', data_buffer_size=10, dataset_impl='raw', ddp_backend='c10d', decoding_format=None, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_port=-1, distributed_rank=0, distributed_world_size=1, distributed_wrapper='DDP', diverse_beam_groups=-1, diverse_beam_strength=0.5, diversity_rate=-1.0, empty_cache_freq=0, eval_bleu=False, eval_bleu_args=None, eval_bleu_detok='space', eval_bleu_detok_args=None, eval_bleu_print_samples=False, eval_bleu_remove_bpe=None, eval_tokenized_bleu=False, fast_stat_sync=False, find_unused_parameters=False, fix_batches_to_gpus=False, fixed_validation_seed=None, forc

| use checkpoint_paths [PosixPath('/content/drive/MyDrive/muss/experiments/fairseq/local_1631567843618/model.pt')]


INFO:fairseq.data.data_utils:loaded 2000 examples from: /tmp/tmpwnzo6byo/tmp.complex-simple.complex
INFO:fairseq.data.data_utils:loaded 2000 examples from: /tmp/tmpwnzo6byo/tmp.complex-simple.simple
INFO:fairseq.tasks.translation:/tmp/tmpwnzo6byo tmp complex-simple 2000 examples
INFO:fairseq_cli.generate:loading model(s) from /content/drive/MyDrive/muss/experiments/fairseq/local_1631567843618/model.pt
INFO:fairseq_cli.generate:NOTE: hypothesis and token scores are output in base 2
INFO:fairseq_cli.generate:Translated 2000 sentences (48384 tokens) in 115.4s (17.33 sentences/s, 419.33 tokens/s)
100%|██████████| 128/128 [5:29:03<00:00, 160.23s/it]

recommended_preprocessors_kwargs={'LengthRatioPreprocessor': {'target_ratio': 0.8362634626138021, 'use_short_name': False}, 'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.7719594617931818, 'use_short_name': False}, 'WordRankRatioPreprocessor': {'target_ratio': 0.8373903360773232, 'language': 'en', 'use_short_name': False}, 'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 0.7178625295578446, 'language': 'en', 'use_short_name': False}, 'GPT2BPEPreprocessor': {}}
